# Dots & Boxes - Decision Theory Project

Within this project, we are going to implement the dots & boxes game by using different techniques learned during this course. The idea is to create the dots & boxes environment and play with a smart agent and a random agent. The agents will simulate the game. Our goal is to make the smart agent more efficient than the smart agent.

## 1. Definition of game
Dots and boxes is a game originally played with pen and paper. The aim of the game is to get more boxes in your possession then your opponent. You start the game with an empty grid. This grid consists of a square (x,y where x and y have the same length) with horizontal dots evenly divided and vertical dots beneath those horizontal dots. So, each dot it in a right angle with every other dot. When You connect four dots you can form a square or in this game called a block. You and your opponent take turns to join up two adjacent dots with a line. If any player forms a box they get a point and they also get to make another move. The player with the most boxes will win. This is not a game of chance; strategies can help a player to win.

## 2. Definition of the environment
### 2.1. States
This game consists of a state space with a grid of 3 by 3. The environment can be in one of these states $S = (S_0, S_1, ..., S_n)$, where n has a a value of 13, which is the end state, because you can maximum only set 2*6 lines in-between dots this is not including the beginning state without any lines. After every turn the program will notify what the current state is and who plays next. This environment consists of:
- The initialisation with:
    - n as the size of the grid
    - hor_links as defining the horizontal dot connections (default all false)
    - ver_links as defining the vertical dot connections (default all false)
    - owners to define the owners of the boxes (default all empty)
    - alphabets to create the x-labels on the grid
    - numbers to create the y-labels on the grid
    - dots consists of every possible dot coordinate on the grid
    - state as all the possible connection coordinates
    - player1 as name for player 1 (default is A)
    - player2 as name for player 2 (default is B)
    - player as the player of the current turn (default is player1)
    - gameOver to indicate if the game has finished
    - rewardsPlayer1 is a list of rewards for player one
    - rewardsPlayer2 is a list of rewards for player two
    - actions consists of all the possible actions in one game (an action will be removed when played)
    - state keeps track of the current state
    - states keeps track of the possible next states.

- The initialisation which starts up a new game 
- The printer which prints the current state space (with part_print as helper)
- show_game which shows the current state of the game
- play_game which does the next action (it can be decided if you want printed output or not.)
- is_game_over which checks if the game is done
- total_rewardsPlayer1_calculator which calculates all the reward together from player 1

In [243]:
import random

def listOfListToTuple(list):
    newList= []
    for i in list:
        newList.append((i[0],i[1]))
    return tuple(newList)


class Reward:
    def __init__(self, name, val):
        self.name = name
        self.val = val
    
    def toString(self):
        return self.name+"|"+str(self.val)

class TransitionProb:
    def __init__(self, Possible_actions):
        self.Possible_actions = Possible_actions
    
    def get_transitionProb(self):
        if len(self.Possible_actions) > 0:
            return 1 / len(self.Possible_actions)
    
    def __str__(self):
        return f"the possible actions are {self.Possible_actions} and the transition probability is {self.get_transitionProb()} "

class DotsAndBoxes:
    def __init__(self, n, player1='A', player2='B', randomPlay=False, printResult=True):
        self.n = n
        self.hor_links = [False] * (n * (n + 1))  # Defining horizontal link connections(now all False)
        self.ver_links = [False] * (n * (n + 1))  # Defining vertical link connections(now all False)
        self.owners = [' '] * (n ** 2)  # defining the owners of created boxes(now blank)
        self.alphabets = list('abcdefghijklmnopqrstuvwxyz')[0:(n + 1)]
        self.numbers = list('0123456789')[0:(n + 1)]
        self.dots = []  # List for points ID
        for num in self.numbers:
            for i in self.alphabets:
                self.dots.append(i + num)
        if randomPlay:
            self.player1 = "random_player"
        else:
            self.player1=player1
        self.player2 = player2
        self.prev_text = ""
        self.player= self.player1
        self.gameOver = False
        self.rewardsPlayer1= []
        self.rewardsPlayer2= []
        self.actions= [["a0","a1"],["a1","a2"],["b0","b1"],["b1","b2"],["c0","c1"],["c1","c2"],["a0","b0"],["b0","c0"],["a1","b1"],["b1","c1"],["a2","b2"],["b2","c2"]]
        self.state=[] #init
        self.states={}
        if randomPlay:
            randomVar= random.choice(self.actions.copy())
            self.play_game(randomVar[0],randomVar[1], printResult)
            

    # part of the following printer function : Helps in same line printing
    def part_print(self, new_text, end=""):
        self.prev_text = self.prev_text + new_text
        if end == "\n":
            print(self.prev_text)
            self.prev_text = ""
        else:
            self.prev_text = self.prev_text + end
        
    # Prints the dots and links and scores in a user friendly manner
    def printer(self, hor_links, ver_links, owners):
        new_hor_links = []
        for i in hor_links:
            if i:
                new_hor_links.append('-------')
            else:
                new_hor_links.append('       ')
        new_ver_links = []
        for i in ver_links:
            if i:
                new_ver_links.append('|       ')
            else:
                new_ver_links.append('        ')
        char = '+'
        hor_index = 0
        ver_index = 0
        owner_index = 0
        row_index = 0
        print('-' * (((self.n + 1) *5) + 8) + '\n')
        print("    a       b       c       d       e       f       g       h       i       j      "[0:((self.n + 1) * 7) + 1] + '\n')
        while True:
            print(" " + str(row_index) + ' ', end=' ')
            for i in range(self.n):
                self.part_print(char, "")
                self.part_print(new_hor_links[hor_index], "")
                hor_index += 1
            self.part_print(char, "\n")
            row_index += 1
            if (hor_index) == len(new_hor_links):
                break
            print("   ", end=' ')
            for i in range(self.n + 1):
                self.part_print(new_ver_links[ver_index], "")
                ver_index += 1
            self.part_print("", "\n")
            ver_index -= (self.n + 1)
            print("   ", end=' ')
            for i in range(self.n):
                if ver_links[ver_index]:
                    self.part_print("|   " + owners[owner_index] + "   ", "")
                else:
                    self.part_print("    " + owners[owner_index] + "   ", "")
                owner_index += 1
                ver_index += 1
            if ver_links[ver_index]:
                self.part_print("|      ", "\n")
            else:
                self.part_print("       ", "\n")
            ver_index += 1
            
        print('\n\n' + '-' * (((self.n + 1) * 5) + 8))
        print("\nscore of player "+ self.player1+" : " + str(self.total_rewardsPlayer1_calculator()))
        print("score of player "+ self.player2+ " : " + str(self.total_rewardsPlayer2_calculator()))
    
    def show_game(self):
        self.printer(self.hor_links, self.ver_links, self.owners)  # prints the boxes
        print(f"It is now the turn of player {self.player}")

    def play_game(self, point1, point2, printResult=True): #play game instead of start
        if point1 != "" and point2 != "":
            pos1 = self.dots.index(point1)
            pos2 = self.dots.index(point2)    
            if self.actions.__contains__([point1, point2]):
                self.state.append([point1,point2])
                box_id = self.create_link(pos1, pos2, self.hor_links, self.ver_links)
                amount_change=0 #amount of boxes that changed owner
                for corner in box_id:
                    if self.change_owner(corner, self.owners, self.player):
                        amount_change+=1
                    
                self.actions.remove([point1,point2])
                if amount_change>0:  # if true the current player will continue the game
                    if self.player == self.player1:
                        reward = Reward(self.player1+ "made a square", amount_change)
                        self.rewardsPlayer1.append(reward)
                        reward = Reward(self.player1+ "made a square", -amount_change)
                        self.rewardsPlayer2.append(reward)
                        if printResult:
                            print(f"{self.player1} got 1 reward")
                    else:
                        reward = Reward(self.player2+" made a square", -amount_change)
                        self.rewardsPlayer1.append(reward)
                        reward = Reward(self.player2+" made a square", amount_change)
                        self.rewardsPlayer2.append(reward)
                        if printResult:
                            print(f"{self.player2} got 1 reward")
                else:
                    self.change_player()
            elif printResult:
                print(f"The points exists already: { self.actions.__contains__([point1, point2])}")
            if printResult:
                self.printer(self.hor_links, self.ver_links, self.owners)  # prints the boxes
                print(f"it is now the turn of player {self.player}")
        if self.is_game_over() & printResult:
            scorep1=self.total_rewardsPlayer1_calculator()
            scorep2= self.total_rewardsPlayer2_calculator()
            print("\nGame over!!")
            if scorep1 < scorep2:
                print("\nplayer "+ self.player2 + " has won the match with " + str(scorep2) + " points")
            elif scorep1 > scorep2:
                print("\nplayer "+ self.player1 + " has won the match with " + str(scorep1) + " points")
            else:
                print("\nthe game is draw!")
    
    def is_game_over(self):
        if ' ' not in self.owners:
            self.gameOver = True
        return self.gameOver
        
    def is_linked(self, pos1, pos2, hor_links, ver_links):
        if pos1 > pos2:
            pos1, pos2 = pos2, pos1
        if (pos1 + 1) % (self.n + 1) == 0 and pos2 % (self.n + 1) == 0:
            return False
        if pos2 - pos1 == self.n + 1:
            return ver_links[pos1]
        elif pos2 - pos1 == 1:
            return hor_links[pos1 - ((pos1 + 1) // (self.n + 1))]
        else:
            return False

    # Checks if the given four points are joined correctly so that a box is formed
    def is_box_completed(self, pos1, pos2, pos3, pos4, hor_links, ver_links):
        all = [pos1, pos2, pos3, pos4]
        all.sort()
        for i in all:
            if i < 0 or i > (((self.n + 1) ** 2) - 1):
                return False
        if (self.is_linked(all[0], all[1], hor_links, ver_links) and self.is_linked(all[2], all[3], hor_links,
                                                                                    ver_links)) and (
                self.is_linked(all[0], all[2], hor_links, ver_links) and self.is_linked(all[1], all[3], hor_links,
                                                                                        ver_links)):
            return True
        else:
            return False

    # checks if the given points are joined and returns a list of topmost left points of the box created .
    # if no box is formed, returns [].
    # raises error if the points cannot be joined !
    def create_link(self, pos1, pos2, hor_links, ver_links):
        e = Exception("Error")
        if self.is_linked(pos1, pos2, hor_links, ver_links):
            raise RuntimeError("already present")
        if pos1 > pos2:
            pos1, pos2 = pos2, pos1
        if (pos1 + 1) % (self.n + 1) == 0 and pos2 % (self.n + 1) == 0:
            raise e
        if pos2 - pos1 == self.n + 1:
            ver_links[pos1] = True
            box_id = []
            check = self.is_box_completed(pos1, pos2, pos1 - 1, pos2 - 1, hor_links, ver_links)
            if check:
                box_id.append(pos1 - 1)
            check = self.is_box_completed(pos1, pos2, pos1 + 1, pos2 + 1, hor_links, ver_links)
            if check:
                box_id.append(pos1)
            return box_id
        elif pos2 - pos1 == 1:
            hor_links[pos1 - ((pos1 + 1) // (self.n + 1))] = True
            box_id = []
            check = self.is_box_completed(pos1, pos2, pos1 - (self.n + 1), pos2 - (self.n + 1), hor_links, ver_links)
            if check:
                box_id.append(pos1 - (self.n + 1))
            check = self.is_box_completed(pos1, pos2, pos1 + (self.n + 1), pos2 + (self.n + 1), hor_links, ver_links)
            if check:
                box_id.append(pos1)
            return box_id
        else:
            raise e

    # removes a link from the given points by making the joining index False in the hor_links or ver_links
    # does nothing if the link is absent
    def remove_link(self, pos1, pos2, hor_links, ver_links):
        e = Exception("Error")
        if pos1 > pos2:
            pos1, pos2 = pos2, pos1
        if (pos1 + 1) % (self.n + 1) == 0 and pos2 % (self.n + 1) == 0:
            raise e
        if (pos2 - pos1) == self.n + 1:
            ver_links[pos1] = False
        elif (pos2 - pos1) == 1:
            hor_links[pos1 - ((pos1 + 1) // (self.n + 1))] = False
        else:
            raise e

    # receives the corner(left topmost point of the box) value and changes its ownership to player name
    def change_owner(self, corner, owners, player):
        if corner != []:
            owners[corner - ((corner + 1) // (self.n + 1))] = player
            return True
        else:
            return False

    # reverses the current player
    def change_player(self):
        if self.player == self.player1:
            self.player = self.player2
        else:
            self.player = self.player1                 
      
                    
    def total_rewardsPlayer1_calculator(self): #R
        result = 0 
        for obj in self.rewardsPlayer1:
            result += obj.val
        return result

    def total_rewardsPlayer2_calculator(self):
        result = 0 
        for obj in self.rewardsPlayer2:
            result += obj.val
        return result
    
    # a random agent function 
    def randomAgent(self, printResult=True):
        for _ in range(len(self.actions)):
            if len(self.actions) > 0:
                randomVar = random.choice(self.actions.copy())
                self.play_game(randomVar[0], randomVar[1],printResult)
    
    def play_random(self, point1=None, point2=None, printResult=True):
        game_player= self.player
        if(point1!=None) | (point2!=None):
            self.play_game(point1,point2,printResult)
            while (self.player!= game_player) & (self.gameOver==False):
                randomVar= random.choice(self.actions.copy())
                self.play_game(randomVar[0],randomVar[1],printResult)
        else:
            while (self.player== game_player) & (self.gameOver==False):
                randomVar= random.choice(self.actions.copy())
                self.play_game(randomVar[0],randomVar[1],printResult)

    def getTransitions(self):
        self.transitions.clear()
        for a in self.actions:
            copy= self.state[:]
            copy.append(a)
            self.transitions.append((1/len(self.actions),copy))
    
    def get_states_from_transitions(self, transitions):
        return (state for prob, state in transitions)
    
    def T(self, state):
        copyA= self.actions[:]
        for a in state:
            first,second=a
            copyA.remove([first,second])
        transitions=[]
        for i in range(len(copyA)):
            state= self.state[:]
            state.append(copyA[i])
            transitions.append((1/len(copyA),listOfListToTuple(state)))
        self.states= self.get_states_from_transitions(transitions)
        return tuple(transitions)

### 2.2 Utility
To know the utility of a game we added a function to get the utility, from a current state and its next action.

In [244]:
def utilily(state, action):
    game= DotsAndBoxes(2)
    for a in state:
        game.play_game(a[0],a[1],False)
    pos1 = game.dots.index(action[0])
    pos2 = game.dots.index(action[1])
    box_id = game.create_link(pos1, pos2, game.hor_links, game.ver_links)
    amount_change=0 #amount of boxes that changed owner
    for corner in box_id:
        if game.change_owner(corner, game.owners, game.player):
            amount_change+=1

    game.actions.remove([action[0], action[1]])
    game.state.append([action[0], action[1]])
    if amount_change>0:
        return amount_change;
    else:
        return 0;

#### 2.2.1 Testing utility

In [245]:
game2 = DotsAndBoxes(2)
game2.play_game("a1", "a2",False)
game2.play_game("b1", "b2",False)
game2.play_game("a1", "b1", False)


utilily(game2.state, ["a2", "b2"])

1

In [246]:
game2 = DotsAndBoxes(2)
game2.play_game("a1", "a2",False)
game2.play_game("b1", "b2",False)
game2.play_game("a1", "b1", False)


utilily(game2.state, ["a0", "b0"])

0

### 2.3. Actions
One action in this game will be making a connection between two dots. Each action from one player is followed by an action from the other player unless the first player has managed to create a box. The actions in this game consist of two coordinates where in between the connection should be formed. These include:

In [247]:
actions= [["a0","a1"],["a1","a2"],["b0","b1"],["b1","b2"],["c0","c1"],["c1","c2"],["a0","b0"],["b0","c0"],["a1","b1"],["b1","c1"],["a2","b2"],["b2","c2"]]
len(actions)

12

These actions include all the possible horizontal and vertical actions. These actions can be activated by dots and boxes play game, where you input the two coordinates and returns the new state with your new action and the algorithms new action. All the actions by the algorithm for the computer play are the functions with comp in front of the name. These include:
- comp_try_box 
- get_comp_turns
- comp_play

### 2.4. Transitions
The transition of $(s'|s,a)$ where s' is the new state, s is the previous state and a represents one of the actions. $s'$ will be decided by the previous state in combination with the action, because it will depend on the new actions what the new stat looks like. The probability $P$ of these transitions will depend on the strategy of the algorithm.

### 2.5. Rewards
The reword of a specific transition, also  will The reword of a specific transition, also $R(s,a,s')$ will depend on how successful the algorithm was. If the algorithm is closer to creating a box, or did create a box, then the reword should be higher then actions that do the opposite. We will work with our own reward system that rewards a player when they form a box and when they win. When the opposite player get a box or a wins, you will get a negative reward.

### 2.6. Policy
The policy will be play steps that will result in the player ending the game with the most amount of boxes. This will result in a high reward. We will find out with Q-learning what this policy is going to look like.

## 3. Testing the environment

In [248]:
game1= DotsAndBoxes(2)
game1.show_game()
for a in actions:
    game1.play_game(a[0],a[1])

-----------------------

    a       b       c 

 0  +       +       +
                            
                           
 1  +       +       +
                            
                           
 2  +       +       +


-----------------------

score of player A : 0
score of player B : 0
It is now the turn of player A
-----------------------

    a       b       c 

 0  +       +       +
    |                       
    |                      
 1  +       +       +
                            
                           
 2  +       +       +


-----------------------

score of player A : 0
score of player B : 0
it is now the turn of player B
-----------------------

    a       b       c 

 0  +       +       +
    |                       
    |                      
 1  +       +       +
    |                       
    |                      
 2  +       +       +


-----------------------

score of player A : 0
score of player B : 0
it is now the turn of player A
-------

### 3.1. Playing against a random opponent
You can play against a random opponent by starting the game and playing with play_random. This function works the same as play_game only when it is the opponents turn it will automatically play a random action. To initialise the game to let the random player go first, you have to indicate in the initialisation that random play is true (this is set to false as default). Like the play_game in play_random and in the initialisation you can also indicate if you want to play the results or not (default is true).

In [249]:
game3 = DotsAndBoxes(2,randomPlay=True, printResult=True)
game3.play_random("a0","a1")

-----------------------

    a       b       c 

 0  +       +       +
                            
                           
 1  +       +-------+
                            
                           
 2  +       +       +


-----------------------

score of player random_player : 0
score of player B : 0
it is now the turn of player B
-----------------------

    a       b       c 

 0  +       +       +
    |                       
    |                      
 1  +       +-------+
                            
                           
 2  +       +       +


-----------------------

score of player random_player : 0
score of player B : 0
it is now the turn of player random_player
-----------------------

    a       b       c 

 0  +       +       +
    |               |       
    |               |      
 1  +       +-------+
                            
                           
 2  +       +       +


-----------------------

score of player random_player : 0
score of pla

In [250]:
game3.play_random("b0","b1")

-----------------------

    a       b       c 

 0  +       +       +
    |       |       |       
    |       |       |      
 1  +       +-------+
                            
                           
 2  +       +       +


-----------------------

score of player random_player : 0
score of player B : 0
it is now the turn of player random_player
-----------------------

    a       b       c 

 0  +       +       +
    |       |       |       
    |       |       |      
 1  +       +-------+
                            
                           
 2  +-------+       +


-----------------------

score of player random_player : 0
score of player B : 0
it is now the turn of player B


In [251]:
game3 = DotsAndBoxes(2)
game3.play_random("a0","a1")

-----------------------

    a       b       c 

 0  +       +       +
    |                       
    |                      
 1  +       +       +
                            
                           
 2  +       +       +


-----------------------

score of player A : 0
score of player B : 0
it is now the turn of player B
-----------------------

    a       b       c 

 0  +       +       +
    |                       
    |                      
 1  +       +       +
            |               
            |              
 2  +       +       +


-----------------------

score of player A : 0
score of player B : 0
it is now the turn of player A


## 4. Random agent playing against random agent
This first example shows the random agent playing against another random agent. In 4.1 we will put this to the test.

In [252]:
game = DotsAndBoxes(2)
game.randomAgent();

-----------------------

    a       b       c 

 0  +       +       +
                            
                           
 1  +-------+       +
                            
                           
 2  +       +       +


-----------------------

score of player A : 0
score of player B : 0
it is now the turn of player B
-----------------------

    a       b       c 

 0  +       +       +
    |                       
    |                      
 1  +-------+       +
                            
                           
 2  +       +       +


-----------------------

score of player A : 0
score of player B : 0
it is now the turn of player A
-----------------------

    a       b       c 

 0  +-------+       +
    |                       
    |                      
 1  +-------+       +
                            
                           
 2  +       +       +


-----------------------

score of player A : 0
score of player B : 0
it is now the turn of player B
-------

### 4.1. Random agent average score
The average of both the random agents playing against each other comes close to zero. This is correct, because the average of one game would be that both random players get 2 boxes which results in two times -1 and two times +1 which results in an end result of 0.

In [265]:
def getRandomGameAverage(aTimes):
    totalPlayer1=0
    totalPlayer2=0
    for _ in range(aTimes):
        game = DotsAndBoxes(2)
        game.randomAgent(False)
        totalPlayer1+= game.total_rewardsPlayer1_calculator()
        totalPlayer2+= game.total_rewardsPlayer2_calculator()
    averageP1= totalPlayer1/aTimes
    averageP2= totalPlayer2/aTimes
    return averageP1,averageP2

getRandomGameAverage(1000)

(-0.098, 0.098)

## 5. Q-learning

In [254]:
from collections import defaultdict
import copy

class QLearningAgent:
    def __init__(self, dab, Ne, Rplus, alpha=None):
        self.gamma = 0.9
        self.all_act = dab.actions
        self.Ne = Ne  # iteration limit in exploration function
        self.Rplus = Rplus  # large value to assign before iteration limit
        self.Q = defaultdict(float)
        self.Nsa = defaultdict(float)
        self.s = None
        self.a = None
        self.r = None

        if alpha:
            self.alpha = alpha
        else:
            self.alpha = lambda n: 1. / (1 + n)  # udacity video
    
    def updateActions(self, actions):
        self.all_act=actions
    
    def f(self, u, n):
        """Exploration function. Returns fixed Rplus until
        agent has visited state, action a Ne number of times.
        Same as ADP agent in book."""
        if n < self.Ne:
            return self.Rplus
        else:
            return u

    def actions_in_state(self,state):
        """Return actions possible in given state.
        Useful for max and argmax."""
        if len(state)==12:
            return [None]
        future_actions=self.all_act[:]
        return listOfListToTuple(future_actions)

    def __call__(self, percept):
        s1, r1 = self.update_state(percept)
        Q, Nsa, s, a, r = self.Q, self.Nsa, self.s, self.a, self.r
        alpha, gamma = self.alpha, self.gamma,
        actions_in_state = self.actions_in_state
        if s1==12:
            Q[s, None] = r1
        if s is not None:
            Nsa[s, a] += 1
            Q[s, a] += alpha(Nsa[s, a]) * (r + gamma * max(Q[s1, a1]
                                                           for a1 in actions_in_state(s1)) - Q[s, a])
        if s1==12:
            self.s = self.a = self.r = None
        else:
            self.s, self.r = s1, r1
            self.a = max(actions_in_state(s1), key=lambda a1: self.f(Q[s1, a1], Nsa[s1, a1]))
        return self.a

    def update_state(self, percept):
        """To be overridden in most cases. The default case
        assumes the percept to be of type (state, reward)."""
        return percept


def run_single_trial(agent_program, dab):
    """Execute trial for given agent_program
    and mdp. mdp should be an instance of subclass
    of mdp.MDP """
    
    current_state = dab.state
    agent_program.all_act=dab.actions
    current_reward = dab.total_rewardsPlayer1_calculator()
    percept = (listOfListToTuple(current_state), current_reward)
    next_action = agent_program(percept)
    if next_action is None:
        raise Exception("The game has ended")
    first,second= next_action
    dab.play_game(first,second,False)


In [280]:
game4 =DotsAndBoxes(2,"smart_agent", "random_agent")
totalAgent=0
totalRandom=0
times=100
q_agent = QLearningAgent(game4, Ne=5, Rplus=2,alpha=lambda n: 60./(59+n) )
for _ in range(times):
    while True:
        if game4.player==game4.player1:
            run_single_trial(q_agent,game4)
        else:
            game4.play_random(printResult=False)
        if game4.gameOver:
            break;
    totalAgent+=game4.total_rewardsPlayer1_calculator()
    totalRandom+= game4.total_rewardsPlayer2_calculator()
    game4 =DotsAndBoxes(2,"smart_agent", "random_agent")
print(totalAgent/times,totalRandom/times)

0.32 -0.32


## 6. Value iteration
We did not manage to complete value iteration.

In [256]:
import copy
def q_value(mdp, s, a, U):
    if not a:
        return utilily(s, a)
    res = 0
    for p, s_prime in mdp.T(s.append(a)):
        res += p * (utilily(s, a) + 0.9 * U[s_prime])
    return res

def value_iteration(mdp, epsilon=0.001):
    """Solving an MDP by value iteration. [Figure 16.6]"""

    U1 = {tuple(s): 0 for s in mdp.states}
    T,gamma = mdp.T(mdp.state), 0.9
    while True:
        U = U1.copy()
        delta = 0
        for s in mdp.states:
            # U1[s] = R(s) + gamma * max(sum(p * U[s1] for (p, s1) in T(s, a))
            #                            for a in mdp.actions(s))
            U1[str(s)] = max(q_value(mdp, s, a, U) for a in mdp.actions)
            delta = max(delta, abs(U1[s] - U[s]))
        if delta <= epsilon * (1 - gamma) / gamma:
            return U

game4 =DotsAndBoxes(2)
utilities= value_iteration(copy.deepcopy(game4))
for state, utility in utilities.items():
    print("{0}:{1:7.4f}".format(state, utility))

AttributeError: 'tuple' object has no attribute 'append'